In [ ]:
import pandas as pd
import numpy as np

# 1. Cargar el DataFrame
df_titanic = pd.read_csv('titanic.csv')

# 2. Eliminación de Columnas Innecesarias o de Texto
# Eliminamos de inmediato todas las columnas que contienen texto o son identificadores.
df_titanic = df_titanic.drop(['PassengerId', 'Name', 'Lname', 'Ticket', 'Cabin'], axis=1)

# 3. Inspección para verificar la eliminación de texto y los nulos restantes
print("\n--- Conteo de No Nulos después de la eliminación de texto ---")
print(df_titanic.info())
# --- IMPUTACIÓN ---

# 1. Rellenar Age (Edad) con la mediana.
mediana_edad = df_titanic['Age'].median()
df_titanic['Age'] = df_titanic['Age'].fillna(mediana_edad)

# 2. Rellenar Embarked (Puerto de Embarque) con la moda.
moda_embarked = df_titanic['Embarked'].mode()[0]
df_titanic['Embarked'] = df_titanic['Embarked'].fillna(moda_embarked)


# --- TRANSFORMACIÓN (Dummies) ---

# 3. Crear variables dummy para Sex y Embarked, eliminando la primera categoría.
df_model = pd.get_dummies(df_titanic, columns=['Sex', 'Embarked'], drop_first=True)

# 4. Crear la variable binaria Is_Child
df_model['Is_Child'] = df_model['Age'] <= 15
df_model['Is_Child'] = df_model['Is_Child'].astype(int) # Convertir True/False a 1/0


# Verificación de que no quede ninguna columna de texto ('object') ni nulos
print("\n--- Verificación Final de Datos Limpios (df_model) ---")
print(df_model.info())

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1. Definición de X e Y
Y = df_model['Survived']
# X son todas las columnas excepto 'Survived'
X = df_model.drop('Survived', axis=1)

# 2. División de Datos
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 3. Entrenar el Modelo de Regresión Logística
print("\nEntrenando Modelo...")
# Se usa 'liblinear' para datasets pequeños o medianos
modelo_logistico = LogisticRegression(random_state=42, solver='liblinear', max_iter=1000)
modelo_logistico.fit(X_train, Y_train)

# 4. Evaluar la Precisión (Accuracy)
Y_pred = modelo_logistico.predict(X_test)
precision = accuracy_score(Y_test, Y_pred) * 100

print(f"\n✅ ¡RESULTADO FINAL! Precisión del modelo de Regresión Logística: {precision:.2f}%")

import pandas as pd

# Extraemos los coeficientes del modelo
coeficientes = modelo_logistico.coef_[0]

# Creamos un DataFrame para visualizarlos de forma clara
df_coeficientes = pd.DataFrame({
    'Característica': X.columns,
    'Coeficiente': coeficientes
})

# Ordenamos por valor absoluto para ver las más influyentes
df_coeficientes['Impacto Absoluto'] = abs(df_coeficientes['Coeficiente'])
df_coeficientes = df_coeficientes.sort_values(by='Impacto Absoluto', ascending=False)

# Imprimimos los resultados
print("\n--- Impacto de cada Característica en la Supervivencia ---")
print(df_coeficientes[['Característica', 'Coeficiente']].round(3))

from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Calcular la Matriz de Confusión
cm = confusion_matrix(Y_test, Y_pred)

# 2. Visualizar la Matriz de Confusión
plt.figure(figsize=(6, 4))
# Título:  (para tu referencia)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predijo Muerte (0)', 'Predijo Supervivencia (1)'],
            yticklabels=['Real Muerte (0)', 'Real Supervivencia (1)'])
plt.title('Matriz de Confusión (Regresión Logística)')
plt.ylabel('Valor Real')
plt.xlabel('Valor Predicho')
plt.show()

# 3. Mostrar el Reporte de Clasificación (Precision, Recall, F1-Score)
print("\n--- Reporte de Clasificación (Métricas Detalladas) ---")
print(classification_report(Y_test, Y_pred, target_names=['Murió (0)', 'Sobrevivió (1)']))

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 1. Entrenar el modelo de Random Forest
print("Entrenando Modelo Random Forest...")
# Usamos 100 árboles (n_estimators) y el mismo random_state para reproducibilidad
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train, Y_train)

# 2. Hacer Predicciones y Evaluar
Y_pred_rf = modelo_rf.predict(X_test)
precision_rf = accuracy_score(Y_test, Y_pred_rf) * 100

print(f"\n✅ Precisión del modelo Random Forest: {precision_rf:.2f}%")

from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Usamos las predicciones hechas por el modelo_rf
cm_rf = confusion_matrix(Y_test, Y_pred_rf)

# 1. Visualizar la Matriz de Confusión del Random Forest
plt.figure(figsize=(6, 4))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Predijo Muerte (0)', 'Predijo Supervivencia (1)'],
            yticklabels=['Real Muerte (0)', 'Real Supervivencia (1)'])
plt.title('Matriz de Confusión (Random Forest)')
plt.ylabel('Valor Real')
plt.xlabel('Valor Predicho')
plt.show() # 
# 2. Mostrar el Reporte de Clasificación
print("\n--- Reporte de Clasificación (Random Forest) ---")
print(classification_report(Y_test, Y_pred_rf, target_names=['Murió (0)', 'Sobrevivió (1)']))

import pandas as pd

# Extraer las puntuaciones de importancia del modelo_rf
importancia_caracteristicas = modelo_rf.feature_importances_

# Crear un DataFrame para la visualización
df_importancia = pd.DataFrame({
    'Característica': X_train.columns,
    'Importancia': importancia_caracteristicas
})

# Ordenar por importancia descendente
df_importancia = df_importancia.sort_values(by='Importancia', ascending=False)

# Imprimir el resultado
print("\n--- Ranking de Importancia (Random Forest) ---")
print(df_importancia.round(4))

import matplotlib.pyplot as plt
import seaborn as sns

# Usaremos el DataFrame de importancia que creaste en el paso anterior
# df_importancia (con las columnas 'Característica' e 'Importancia')

plt.figure(figsize=(10, 6))
# Crear un gráfico de barras ordenado por la Importancia
sns.barplot(x='Importancia', y='Característica', data=df_importancia, palette='viridis')

plt.title('Importancia de las Características (Random Forest)')
plt.xlabel('Puntuación de Importancia (Suma = 1.0)')
plt.ylabel('Característica')
plt.show() #